This code is the dummy code that:

Input: 
- random "fidelity" F from 0 to 1
- property P that was calculated from F

Output:
- Macchine learned `model` that given P output a predicted F

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# -----------------------------
# Dummy function: compute 3 probabilities from fidelity F
# -----------------------------
def compute_P(F):
    p1 = F + 1
    p2 = np.exp(F)
    p3 = F**2
    return np.stack([p1, p2, p3], axis=-1)

# -----------------------------
# Generate dataset
# -----------------------------
N = 10000
F = np.random.rand(N)  # fidelity in [0,1]
P = compute_P(F)       # corresponding probabilities


Epoch 10, Loss: 0.000021
Epoch 20, Loss: 0.000030
Epoch 30, Loss: 0.000019
Epoch 40, Loss: 0.000007
Epoch 50, Loss: 0.000006
True F: [0.2 0.5 0.8]
Predicted F: [0.2051563  0.50096    0.79891676]


In [ ]:
# Convert to PyTorch tensors
X = torch.tensor(P, dtype=torch.float32)   # inputs: 3 probabilities
y = torch.tensor(F, dtype=torch.float32).unsqueeze(1)  # output: fidelity

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# -----------------------------
# Simple MLP model
# -----------------------------
model = nn.Sequential(
    nn.Linear(3, 32),
    nn.ReLU(),
    nn.Linear(32, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# -----------------------------
# Training loop
# -----------------------------
epochs = 50
for epoch in range(epochs):
    for xb, yb in dataloader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.6f}")


In [ ]:

# -----------------------------
# Test prediction
# -----------------------------
test_F = np.array([0.2, 0.5, 0.8])
test_P = torch.tensor(compute_P(test_F), dtype=torch.float32)
pred_F = model(test_P).detach().numpy()
print("True F:", test_F)
print("Predicted F:", pred_F.flatten())


In [4]:
test_F = np.array([0.1, 0.4, 0.7])
test_P = torch.tensor(compute_P(test_F), dtype=torch.float32)
pred_F = model(test_P).detach().numpy()
print("True F:", test_F)
print("Predicted F:", pred_F.flatten())

True F: [0.1 0.4 0.7]
Predicted F: [0.09789059 0.40079838 0.7039053 ]
